In [1]:
from wtforms import Form

from wtforms.ext.sqlalchemy.orm import model_form

from dash_deep.app import scripts_db_models
from dash_deep.widjets.widjets_factory import generate_widjet_from_form
from dash_deep.utils import generate_wtform_instances_and_input_form_widjets

wtform_classes, scripts_input_form_widjets = generate_wtform_instances_and_input_form_widjets(scripts_db_models)



# register button, validate, spawn a thread with the job (but job still has to be defined)

# register this widjets now?

# you have endpoints

In [7]:
instance = wtform_classes[0]

In [11]:
instance.__class__.__name__

'PascalSegmentationForm'

In [4]:
for i in wtform_classes[0]:
    
    print(i)

<input id="batch_size" name="batch_size" type="text" value="">
<input id="learning_rate" name="learning_rate" type="text" value="">
<input id="output_stride" name="output_stride" type="text" value="">


In [2]:
scripts_db_models

[dash_deep.scripts.semantic_segmentation_train_script.PascalSegmentation]

In [4]:
form_instance =  wtform_classes[0]()

In [6]:
form_instance.__class__

wtforms.ext.sqlalchemy.orm.PascalSegmentationForm

In [1]:
from wtforms import Form, BooleanField, StringField,  IntegerField, validators


class RegistrationForm(Form):
    
    username = StringField('Username', [validators.Length(min=4, max=25)])
    email = StringField('Email Address', [validators.Length(min=6, max=35)])
    level = IntegerField('User Level', [validators.NumberRange(min=0, max=10)])

In [12]:
test = RegistrationForm()
test.validate()

False

In [14]:
test.username.data = 'Daniil'
test.validate()

False

In [18]:
for i in test:
    i.data = 'hi there'

In [22]:
test.username.data

'hi there'

In [23]:
zip([1, 2, 3], test)

[(1, <wtforms.fields.core.StringField at 0x7f8294639ad0>),
 (2, <wtforms.fields.core.StringField at 0x7f82945df150>),
 (3, <wtforms.fields.core.IntegerField at 0x7f82945df450>)]

In [15]:
test.errors

{'email': [u'Field must be between 6 and 35 characters long.'],
 'level': [u'Number must be between 0 and 10.']}

In [11]:
test.level.data

'4'

In [7]:
for i in test:
    
    print(i)

<input id="username" name="username" type="text" value="">
<input id="email" name="email" type="text" value="">
<input id="level" name="level" type="text" value="">


In [57]:
test.username.data = 'Daniil'
test.email.data = 'dan@gmail.com'
test.validate()
test.errors

{}

In [37]:
another = RegistrationForm(username='Danii', email='dan@gmail.com', accept_rules=True)

In [39]:
another.username.data
another.validate()

False

In [40]:
another.errors

{'accept_rules': [u'This field is required.']}

In [51]:
test.validate()

False

In [52]:
test.errors

{'email': [u'Field must be between 6 and 35 characters long.'],
 'username': [u'Field must be between 4 and 25 characters long.']}